In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics


In [2]:
#import the data
url = 'https://raw.githubusercontent.com/jamesdinardo/Retail-Forecasting/master/final_df.csv'
df = pd.read_csv(url)

SyntaxError: invalid syntax (<ipython-input-2-923964d563d3>, line 2)

In [5]:
print('Shape of final dataframe: {}'.format(df.shape))
df.head()

Shape of final dataframe: (421570, 16)


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales,Type,Size
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,24924.50,A,151315
1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,2,50605.27,A,151315
2,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,3,13740.12,A,151315
3,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,4,39954.04,A,151315
4,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,5,32229.38,A,151315


In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  category      
 1   Date          421570 non-null  datetime64[ns]
 2   Temperature   421570 non-null  float64       
 3   Fuel_Price    421570 non-null  float64       
 4   MarkDown1     150681 non-null  float64       
 5   MarkDown2     111248 non-null  float64       
 6   MarkDown3     137091 non-null  float64       
 7   MarkDown4     134967 non-null  float64       
 8   MarkDown5     151432 non-null  float64       
 9   CPI           421570 non-null  float64       
 10  Unemployment  421570 non-null  float64       
 11  IsHoliday     421570 non-null  bool          
 12  Dept          421570 non-null  category      
 13  Type          421570 non-null  object        
 14  Size          421570 non-null  int64         
 15  Weekly_Sales  421

In [ ]:
#convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

#convert Store, Dept, and Type columns to category
df['Store'] = df['Store'].astype('category')

df['Dept'] = df['Dept'].astype('category')

df['Type'] = df['Type'].astype('category')

df.info()

Testing the following KNN models:


1. KNN trained on all data

2. KNN with date columns discarded

3. KNN with only lagged values of weekly sales as features

4. KNN with all features plus lagged values of weekly sales

1. KNN trained on all data

In [11]:
#create separate features for Week, Month, and Year
df['Week'] = df['Date'].dt.week
df['Week'] = df['Week'].astype('category')

df['Month'] = df['Date'].dt.month
df['Month'] = df['Month'].astype('category')

df['Year'] = df['Date'].dt.year
df['Year'] = df['Year'].astype('category')

In [12]:
df.drop('Date', axis=1, inplace=True)

In [13]:
df_dummies = pd.get_dummies(df, drop_first=True)

In [14]:
df_dummies.columns

Index(['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday',
       ...
       'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
       'Month_11', 'Month_12', 'Year_2011', 'Year_2012'],
      dtype='object', length=202)

In [15]:
y = df_dummies['Weekly_Sales'].values.reshape(-1, 1)
X = df_dummies.drop('Weekly_Sales', axis=1).values

print(y.shape)
print(X.shape)

(421570, 1)
(421570, 201)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape)
print(X_test.shape)

(316177, 201)
(105393, 201)


In [17]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)

KNeighborsRegressor()

In [18]:
y_pred = knn.predict(X_test)

print('R2: {}'.format(knn.score(X_test, y_test)))
print('RMSE: {}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

R2: -0.07634168010919318
RMSE: 23709.109148747408


KNN is unable to achieve good results with the dummy variables

2. KNN model with date columns discarded

In [35]:
df_no_date = df.drop(['Week', 'Month', 'Year'], axis=1)
df_no_date_dummies = pd.get_dummies(df_no_date, drop_first=True)

In [36]:
y = df_no_date_dummies['Weekly_Sales'].values.reshape(-1, 1)
X = df_no_date_dummies.drop('Weekly_Sales', axis=1).values

print(y.shape)
print(X.shape)

(421570, 1)
(421570, 132)


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape)
print(X_test.shape)

(316177, 132)
(105393, 132)


In [38]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)

KNeighborsRegressor()

In [39]:
print('R2: {}'.format(knn.score(X_test, y_test)))

R2: 0.34244460269834787


In [57]:
y_pred = knn.predict(X_test)
print('RMSE: {}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

RMSE: 14605.529306408394


3. KNN with only lagged values of weekly sales as features

In [206]:
df_date_only = df[['Date', 'Weekly_Sales']]
print(df_date_only.shape)
df_date_only.head()

(421570, 2)


,Date,Weekly_Sales
0,2010-05-02,24924.50
1,2010-05-02,50605.27
2,2010-05-02,13740.12
3,2010-05-02,39954.04
4,2010-05-02,32229.38


In [207]:
df_date_only.set_index('Date', inplace=True)

In [208]:
df_date_only.head()

,Weekly_Sales
Date,
2010-05-02,24924.50
2010-05-02,50605.27
2010-05-02,13740.12
2010-05-02,39954.04
2010-05-02,32229.38


In [209]:
#group by date to get best selling weeks across all stores
date_only_grouped = df_date_only.groupby('Date').sum()
date_only_grouped.head()

,Weekly_Sales
Date,
2010-01-10,42239875.87
2010-02-04,50423831.26
2010-02-07,48917484.50
2010-02-19,48276993.78
2010-02-26,43968571.13


In [210]:
#attempt to convert this time series into a supervised learning problem
#this function is essentially a more customizable way of using the Pandas shift method,
#something like: df['t-1'] = df['t'].shift(1)

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """Takes a time series as argument and returns a dataframe for supervised learning
    
    Arguments: data: Sequence of observations as a list or NumPy array
    n_in: Number of lag observations as input
    n_out: Number of observations as output
    dropnan: whether or not to drop rows with NaN values"""

    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
        
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    #drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [214]:
display(df_date_only)
display(series_to_supervised(df_date_only))

,Weekly_Sales
Date,
2010-05-02,24924.50
2010-05-02,50605.27
2010-05-02,13740.12
2010-05-02,39954.04
2010-05-02,32229.38
...,...
2012-10-26,2487.80
2012-10-26,5203.31
2012-10-26,56017.47


,var1(t-1),var1(t)
Date,,
2010-05-02,24924.50,50605.27
2010-05-02,50605.27,13740.12
2010-05-02,13740.12,39954.04
2010-05-02,39954.04,32229.38
2010-05-02,32229.38,5749.03
...,...,...
2012-10-26,54608.75,2487.80
2012-10-26,2487.80,5203.31
2012-10-26,5203.31,56017.47


In [265]:
supervised_date_only = series_to_supervised(df_date_only, 3)
X = supervised_date_only.drop('var1(t)', axis=1).values
y = supervised_date_only['var1(t)'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape)
print(X_test.shape)

(316175, 3)
(105392, 3)


In [266]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.4403229082474577

In [275]:
display(date_only_grouped)
supervised_date_only_grouped = series_to_supervised(date_only_grouped, 2)
display(supervised_date_only_grouped)

,Weekly_Sales
Date,
2010-01-10,42239875.87
2010-02-04,50423831.26
2010-02-07,48917484.50
2010-02-19,48276993.78
2010-02-26,43968571.13
...,...
2012-10-08,47403451.04
2012-10-19,45122410.57
2012-10-26,45544116.29


,var1(t-2),var1(t-1),var1(t)
Date,,,
2010-02-07,42239875.87,50423831.26,48917484.50
2010-02-19,50423831.26,48917484.50,48276993.78
2010-02-26,48917484.50,48276993.78,43968571.13
2010-03-09,48276993.78,43968571.13,47194257.61
2010-03-12,43968571.13,47194257.61,49909027.88
...,...,...,...
2012-10-08,43734899.40,50009407.92,47403451.04
2012-10-19,50009407.92,47403451.04,45122410.57
2012-10-26,47403451.04,45122410.57,45544116.29


In [276]:
X = supervised_date_only_grouped.drop('var1(t)', axis='columns').values
y = supervised_date_only_grouped['var1(t)'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape)
print(X_test.shape)

(105, 2)
(36, 2)


In [277]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.04161368207709559

4. KNN with all features plus lagged values of weekly sales

Grouping by date seems to perform worse. So let's add lagged observations to the remainder of the dataframe

In [278]:
#Combine this information with the rest of the dataframe
supervised_date_only = series_to_supervised(df_date_only, dropnan=False)
len(supervised_date_only)


421570

In [291]:
df_lag_no_date = df.drop('Date', axis='columns')
df_lag_no_date['Lagged Sales'] = supervised_date_only['var1(t-1)'].values
df_lag_no_date.dropna(inplace=True)
df_lag_no_date.head()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Type,Size,Weekly_Sales,Lagged Sales
1,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,2,A,151315,50605.27,24924.50
2,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,3,A,151315,13740.12,50605.27
3,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,4,A,151315,39954.04,13740.12
4,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,5,A,151315,32229.38,39954.04
5,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,6,A,151315,5749.03,32229.38


In [292]:
y = df_lag_no_date['Weekly_Sales'].values.reshape(-1, 1)
X = df_lag_no_date.drop('Weekly_Sales', axis='columns').values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape)
print(X_test.shape)

(316176, 15)
(105393, 15)


In [301]:
df_lag_no_date = pd.get_dummies(df_lag_no_date)
df_lag_no_date.head()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Size,Weekly_Sales,Lagged Sales,Type_A,Type_B,Type_C
1,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,2,151315,50605.27,24924.50,1,0,0
2,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,3,151315,13740.12,50605.27,1,0,0
3,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,4,151315,39954.04,13740.12,1,0,0
4,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,5,151315,32229.38,39954.04,1,0,0
5,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,6,151315,5749.03,32229.38,1,0,0


In [302]:
test_lag = df_lag_no_date.drop(['Type_A', 'Type_B', 'Type_C'], axis=1)

In [304]:
y = test_lag['Weekly_Sales'].values.reshape(-1, 1)
X = test_lag.drop('Weekly_Sales', axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape)
print(X_test.shape)

(316176, 14)
(105393, 14)


In [308]:
df_lag_no_date.head()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Size,Weekly_Sales,Lagged Sales,Type_A,Type_B,Type_C
1,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,2,151315,50605.27,24924.50,1,0,0
2,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,3,151315,13740.12,50605.27,1,0,0
3,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,4,151315,39954.04,13740.12,1,0,0
4,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,5,151315,32229.38,39954.04,1,0,0
5,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,6,151315,5749.03,32229.38,1,0,0


In [305]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.19877015674215737

In [242]:
series_to_supervised(date_only_grouped, 1)

,var1(t-1),var1(t)
Date,,
2010-02-04,42239875.87,50423831.26
2010-02-07,50423831.26,48917484.50
2010-02-19,48917484.50,48276993.78
2010-02-26,48276993.78,43968571.13
2010-03-09,43968571.13,47194257.61
...,...,...
2012-10-08,50009407.92,47403451.04
2012-10-19,47403451.04,45122410.57
2012-10-26,45122410.57,45544116.29
